## Working With Drug Data From the ChEMBL Database

When working on drug discovery projects, it's handy to have access to a set of chemical structures and associated data for marketed drugs. If you're considering introducing new functionality, someone invariably asks whether that functionality has been used in a marketed drug. It's also helpful to compare the properties of a new compound or compounds to those of marketed drugs. Early in my career, I remember a new medicinal chemist asking Josh Boger, the founder of Vertex Pharmaceuticals, what they should do on their first day of work. Boger responded, "read the Merck Index, so you can see what a drug is supposed to look like". Recently a [few](https://pubs.acs.org/doi/10.1021/acs.jmedchem.8b00686) [papers](https://www.nature.com/articles/s41570-022-00451-0) have been published showing how the properties of drugs have changed over time. I thought it might be helpful to create a notebook showing how to extract and clean drug data from ChEMBL and use it to perform a similar analysis.

### Overview

In this notebook we'll perform the following steps.

1. Download the ChEMBL database
2. Query ChEMBL for drug data
3. Remove duplicates from the ChEMBL data
4. Divide the ChEMBL data into three groups based on the first approval date, before 1997, 1997-2017, after 2017
5. Compare the molecular weight and calculated logp distributions for the three groups and determine if the differences between groups are statistically significant.

In [1]:
import re
import chembl_downloader
import pandas as pd
import scikit_posthocs as sp
import seaborn as sns
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.rdBase import BlockLogs
from tqdm.auto import tqdm

Enable progress bars in Pandas

In [2]:
tqdm.pandas()

### 1. Download the ChEMBL database

We begin by using the awesome [ChEMBL downloader](https://github.com/cthoyt/chembl-downloader) by Charles Tapley Hoyt to download the latest version of the ChEMBL database.  On my laptop, this took 7 minutes and consumed 27GB of disk space. The ChEMBL downloader not only makes it easy to download the database, it also allows you to submit queries and returns the results as a Pandas dataframe. 

In [ ]:
path = chembl_downloader.download_extract_sqlite()

In [ ]:
path

Define SQL to extract drug data from ChEMBL. It would probably be useful to explain the query below.  We're joining three tables:
- molecule_dictionary - Table storing a non-redundant list of curated compounds for ChEMBL (includes preclinical compounds, drugs and clinical candidate drugs) and some associated attributes.
- compound_structures - Table storing various structure representations (e.g., Molfile, InChI) for each compound
- compound properties - Table storing calculated physicochemical properties for compounds, now calculated with RDKit and ChemAxon software (note all but FULL_MWT and FULL_MOLFORMULA are calculated on the parent structure)


We select records where
- max_phase = 4 (approved drugs)
- molecule_type = Small molecule
- molecular weight is between 200 and 1000

The group by ensures that we have one record for each canonical_smiles, molregno pair

In [ ]:
sql = """
select cs.canonical_smiles, cs.molregno, pref_name, first_approval, dosed_ingredient, oral, parenteral, topical,
       black_box_warning, first_in_class from molecule_dictionary md
join compound_structures cs on cs.molregno = md.molregno
join compound_properties cp on md.molregno = cp.molregno
where max_phase = 4 and molecule_type = 'Small molecule'
and cp.full_mwt > 200 and cp.full_mwt < 1000
group by cs.canonical_smiles, cs.molregno
"""

In [ ]:
df = chembl_downloader.query(sql)

In [ ]:
df

### 3. Remove duplicates from the ChEMBL data

Several drugs are in ChEMBL multiple times as different salt forms.  To simplify our analysis, we'd like to only have each drug represented once.  We can use the MolStandardize functionality in the RDKit to remove salts and add another column with the standardized SMILES (std_smiles).  I used the example code below from the bitsilla blog to standardize the SMILES. 

In [ ]:
# Borrowed from https://bitsilla.com/blog/2021/06/standardizing-a-molecule-using-rdkit/
def standardize(smiles):
    # follows the steps in
    # https://github.com/greglandrum/RSC_OpenScience_Standardization_202104/blob/main/MolStandardize%20pieces.ipynb
    # as described **excellently** (by Greg) in
    # https://www.youtube.com/watch?v=eWTApNX8dJQ
    mol = Chem.MolFromSmiles(smiles)

    # removeHs, disconnect metal atoms, normalize the molecule, reionize the molecule
    clean_mol = rdMolStandardize.Cleanup(mol)

    # if many fragments, get the "parent" (the actual mol we are interested in) 
    parent_clean_mol = rdMolStandardize.FragmentParent(clean_mol)

    # try to neutralize molecule
    uncharger = rdMolStandardize.Uncharger()  # annoying, but necessary as no convenience method exists
    uncharged_parent_clean_mol = uncharger.uncharge(parent_clean_mol)

    # note that no attempt is made at reionization at this step
    # nor at ionization at some pH (rdkit has no pKa caculator)
    # the main aim to represent all molecules from different sources
    # in a (single) standard way, for use in ML, catalogue, etc.

    te = rdMolStandardize.TautomerEnumerator()  # idem
    taut_uncharged_parent_clean_mol = te.Canonicalize(uncharged_parent_clean_mol)

    return Chem.MolToSmiles(taut_uncharged_parent_clean_mol)

The code below performs the standardization and creates a new column.  Note the use of [BlockLogs](https://rdkit.org/docs/source/rdkit.rdBase.html).  The standardizer has a lot of logging messages that I prefer to ignore.   

In [ ]:
with BlockLogs():
    df['std_smiles'] = df.canonical_smiles.progress_apply(standardize)

Drop any rows that don't have a first_approval year. 

In [ ]:
df_ok = df.dropna(subset=["first_approval"]).copy()

The **first_approval** field comes over from ChEMBL as a floating point number.  This bugs me, so I'll convert it to an integer.

In [ ]:
df_ok.first_approval = df_ok.first_approval.astype(int)
df_ok

Let's take a look a structures that occur multiple times.  The first one is citrate. This is an odd case where we have things like sodium citrate where the salt is larger than the parent.  There aren't a lot of these and I don't find them interesting so I'm ignoring them.  The second example, which occurs six times is more interesting. 

In [ ]:
df_freq = uru.value_counts_df(df_ok, "std_smiles")
df_freq

If we take a look at this one, we see various forms of penicillin.

In [ ]:
query_smi = df_freq.std_smiles.values[1]
print(query_smi)
df_ok.query("std_smiles == @query_smi").sort_values("first_approval")

Now let's remove the duplicate structures. To do this, we sort by **first_approval** then remove duplicate **std_smiles**"

In [ ]:
df_ok_nodupe = df_ok.sort_values("first_approval").drop_duplicates("std_smiles").copy()
len(df_ok), len(df_ok_nodupe)

Next, let's limit our analysis to oral drugs. 

In [ ]:
df_oral_ok_nodupe = df_ok_nodupe.query("oral == 1").copy()
len(df_oral_ok_nodupe)

There still may be duplicates that are the same structure with different charge states.  One way to handle this is to generate an InChI for each structure and remove the charge layer.  In this way, the charged and uncharged versions will have the same InChI string. 

Define a function to generate an InChI without the charge field.  This is a bit of a hack.  I should probably be passing the "nochg" option to Chem.MolToInchi but I can't figure out how to do that. 

In [ ]:
def smi_to_inchi_nochg(smi):
    mol = Chem.MolFromSmiles(smi)
    inchi = Chem.MolToInchi(mol)
    return re.sub("/p\+[0-9]+", "", inchi)

To test our function we'll generate InChi strings for the neutral and protonated forms of propylamine.  We can then check to see if the InChi strings are the same.

In [ ]:
no_chg_inchi = smi_to_inchi_nochg("CCCN")
chg_inchi = smi_to_inchi_nochg("CCC[NH3+]")
no_chg_inchi, chg_inchi, chg_inchi == no_chg_inchi

Now we'll apply this function to all the structures in our dataframe. As above, I'm using **BlockLogs** to ignore the logging messages.

In [ ]:
with BlockLogs():
    df_oral_ok_nodupe['inchi'] = df_oral_ok_nodupe.std_smiles.apply(smi_to_inchi_nochg)

Let's see if we have any InChI duplicates.  We can use the function **value_counts_df** from the [useful_rdkit_utils](https://github.com/PatWalters/useful_rdkit_utils) package to convert the results of the Pandas value_counts method to a nicely formatted dataframe.  It looks like there's only one example with two different charge states.

In [ ]:
df_dupe_inchi = uru.value_counts_df(df_oral_ok_nodupe, "inchi")
df_dupe_inchi

Let's take a closer look at this example.

In [ ]:
inchi_val = df_dupe_inchi.inchi.values[0]
dupe_inchi_df = df_oral_ok_nodupe.query("inchi == @inchi_val")
dupe_inchi_df

In [ ]:
dupe_inchi_df.std_smiles.values

In [ ]:
dupe_mol_list = [Chem.MolFromSmiles(x) for x in dupe_inchi_df.std_smiles]
MolsToGridImage(dupe_mol_list, useSVG=True, subImgSize=(350, 350))

In order to clean things up a bit, we can drop the duplicate record. 

In [ ]:
df_final_drug = df_oral_ok_nodupe.sort_values("first_approval").drop_duplicates("inchi").copy()
len(df_final_drug)

### 4. Divide the ChEMBL data into three groups, before 1997, 1997-2017, after 2017

Now that we have a clean datset, we can do some analysis.  We'll start dividing the data into three sets based on the first approval year.  To do this we'll use the criteria defined in a 2023 paper by [Hartung, Webb, and Crespo](https://www.nature.com/articles/s41570-022-00451-0). Note how the Pandas cut function makes it easy to bin the data. 

In [ ]:
df_final_drug['era'] = pd.cut(df_final_drug.first_approval, [0, 1996, 2017, 5000],
                              labels=["before 1997", "1997-2017", "after 2017"])

In [ ]:
df_final_drug

### 5. Compare the molecular weight and calculated logp distributions for the three groups, and determine if the differences between groups are statistically significant.
Now let's calculate the parameters that define Lipinski's Rule of 5.  Fortunately, the [useful_rdkit_utils](https://github.com/PatWalters/useful_rdkit_utils) package has a convenience function to make this easy.

In [ ]:
ro5_calc = uru.Ro5Calculator()
df_final_drug[ro5_calc.names] = df_final_drug.std_smiles.apply(ro5_calc.calc_smiles).tolist()

In [ ]:
df_final_drug

With that data in hand, we can make boxplots to show the molecular weight distributions over the three time periods in question.  Based on the boxplots, it appears that there is a trend toward increasing molecular weight over time.  However, we also want to look at whether there is a statistically signficant difference between the distributions. We'll look at this below. 

In [ ]:
ax = sns.boxplot(x="era", y="MolWt", data=df_final_drug)
ax.set_xlabel("Era");

As mentioned above, we want to evaluate whether the molecular weight distributions above are different.  When working with normally distributed data, we would use something like Student's t-test to compare two distributions.  Since the distributions we're dealing with are not normally distributed we'll use the non-parametric Wilcoxon Rank Sum Test.  We're dealing with three distributions, so we need to correct the p-value to account for multiple comparisons.  Fortunately for us, there's the **scikit-posthocs** Python package to do the heavy lifting.  For more on multiple comparisons and post-hoc tests, please see this Practical Cheminformatics blog post. Looking at the plot below, we see that we can invalidate the null hypothesis that the means of the distributions are the same with at least p < 0.01.

In [ ]:
sns.set(rc={'figure.figsize': (8, 6)}, font_scale=1.5)
pc = sp.posthoc_mannwhitney(df_final_drug, val_col="MolWt", group_col="era", p_adjust='holm')
heatmap_args = {'linewidths': 0.25, 'linecolor': '0.5', 'clip_on': False, 'square': True,
                'cbar_ax_bbox': [0.80, 0.35, 0.04, 0.3]}
_ = sp.sign_plot(pc, **heatmap_args)

Following the pattern above, we can look do the same analysis with the calculated LogP.  In this case, the two distributions on the right look similar.  Let's look at the statistics and see if they are different. 

In [ ]:
ax = sns.boxplot(x="era", y="LogP", data=df_final_drug)
ax.set_xlabel("Era");

Again, we'll use scikit-posthocs to create a heatmap.  In this case we can see that for "1997-2017" and "after 2017" sets, we cannot invalidate the null hypothesis that that distributions are the same. 

In [ ]:
sns.set(rc={'figure.figsize': (8, 6)}, font_scale=1.5)
pc = sp.posthoc_mannwhitney(df_final_drug, val_col="LogP", group_col="era", p_adjust='holm')
heatmap_args = {'linewidths': 0.25, 'linecolor': '0.5', 'clip_on': False, 'square': True,
                'cbar_ax_bbox': [0.80, 0.35, 0.04, 0.3]}
_ = sp.sign_plot(pc, **heatmap_args)

### Acknowledgements

I'd like to thank Emanuele Perola for motivating this notebook and Brian Kelley and Joann Prescott-Roy for helpful discussions.